# Building a Simple RAG System

## Retrieval-Augmented Generation Exercise

In this exercise, you'll build a RAG system that eliminates AI hallucinations by grounding responses in real product data.

### Learning Objectives
- Understand how RAG prevents hallucinations
- Implement keyword-based retrieval
- See the impact of context on AI responses
- Compare accuracy with and without RAG

## Part 1: Setup and Load Mock Data

In [ ]:
# Mock product catalog with intentionally distinct keywords
products = [
    {
        "name": "TurboCache Pro",
        "description": "Lightning-fast in-memory caching solution with sub-millisecond latency",
        "features": ["10GB capacity", "LRU eviction", "distributed mode", "Redis compatible"],
        "keywords": ["speed", "fast", "performance", "cache", "memory", "quick", "turbo"],
        "price": "$99/month"
    },
    {
        "name": "SecureVault Enterprise",
        "description": "Military-grade encryption for sensitive data protection",
        "features": ["AES-256 encryption", "biometric auth", "SOC2 compliant", "key rotation"],
        "keywords": ["security", "encryption", "protection", "vault", "safe", "secure", "privacy"],
        "price": "$199/month"
    },
    {
        "name": "CloudSync Manager",
        "description": "Real-time data synchronization across cloud platforms",
        "features": ["Multi-cloud support", "version control", "1TB storage", "automatic backups"],
        "keywords": ["sync", "cloud", "backup", "storage", "synchronization", "replication"],
        "price": "$149/month"
    },
    {
        "name": "DataFlow Analytics",
        "description": "Stream processing and real-time analytics platform",
        "features": ["Apache Kafka integration", "ML pipelines", "custom dashboards", "alerting"],
        "keywords": ["analytics", "data", "streaming", "metrics", "insights", "dashboard"],
        "price": "$299/month"
    }
]

print(f"Loaded {len(products)} products")
for p in products:
    print(f"  - {p['name']}")

## Part 2: Query Without RAG (Observe Hallucination)

First, let's see what happens when AI doesn't have access to our product information.

In [ ]:
def ask_ai_without_context(question):
    """Simulate AI response without any context"""
    # In real implementation, this would call OpenAI/Claude API
    # For demo, we'll show typical hallucinated response
    
    if "TurboCache" in question:
        return """
Based on my knowledge, TurboCache Pro includes:
- Advanced ML-based caching algorithms
- Automatic scaling to 100TB
- Built-in blockchain verification
- Quantum-resistant encryption
- Free tier available

Note: These features are likely made up since I don't have real product information.
"""
    else:
        return "I don't have specific information about that product."

# Test query without RAG
response = ask_ai_without_context("What features does TurboCache Pro have?")
print("WITHOUT RAG (Hallucinated Response):")
print("=" * 50)
print(response)

## Part 3: Build Simple Keyword Search

Now let's implement a basic retrieval system using keyword matching.

In [ ]:
def search_products(query, products, threshold=1):
    """
    Simple keyword search - returns products matching query terms
    
    Args:
        query: User's search query
        products: List of product dictionaries
        threshold: Minimum number of keyword matches required
    
    Returns:
        List of matching products
    """
    # Convert query to lowercase and split into words
    query_words = query.lower().split()
    
    matches = []
    for product in products:
        # Count how many query words match product keywords
        match_count = sum(
            1 for word in query_words 
            if word in product['keywords']
        )
        
        # Include product if it meets threshold
        if match_count >= threshold:
            matches.append({
                'product': product,
                'relevance': match_count
            })
    
    # Sort by relevance (most matches first)
    matches.sort(key=lambda x: x['relevance'], reverse=True)
    
    return [m['product'] for m in matches]

# Test the search function
test_queries = [
    "fast performance",
    "security encryption",
    "cloud backup",
    "data analytics"
]

for query in test_queries:
    results = search_products(query, products)
    print(f"Query: '{query}'")
    print(f"Found: {[r['name'] for r in results]}")
    print()

## Part 4: Create Context from Search Results

Format the search results into context that can be injected into the AI prompt.

In [ ]:
def create_context(search_results):
    """Format search results into context for the AI"""
    if not search_results:
        return "No product information available."
    
    context = "Product Information:\n\n"
    for product in search_results:
        context += f"Product: {product['name']}\n"
        context += f"Description: {product['description']}\n"
        context += f"Features: {', '.join(product['features'])}\n"
        context += f"Price: {product['price']}\n\n"
    
    return context

# Example: Create context for "fast cache" query
results = search_products("fast cache", products)
context = create_context(results)
print("CONTEXT TO INJECT:")
print("=" * 50)
print(context)

## Part 5: Query With RAG (Accurate Response)

Now let's combine search and context to provide accurate, grounded responses.

In [ ]:
def ask_ai_with_rag(question, products):
    """Use RAG to provide accurate response"""
    
    # Step 1: Search for relevant products
    search_results = search_products(question, products)
    
    # Step 2: Create context from results
    context = create_context(search_results)
    
    # Step 3: Build augmented prompt
    prompt = f"""Based on the following product information:

{context}

Question: {question}

Answer based only on the provided information. If the information doesn't answer the question, say so."""
    
    # In real implementation, send this prompt to OpenAI/Claude
    # For demo, we'll create an accurate response based on context
    
    if not search_results:
        return "I couldn't find any products matching your query. Please try different search terms."
    
    # Simulate accurate response based on actual data
    if "TurboCache" in question and search_results:
        return f"""Based on the provided information, TurboCache Pro includes:
- {', '.join(search_results[0]['features'])}
- Price: {search_results[0]['price']}

These are the actual features from our product catalog."""
    
    # Generic response showing found products
    product_names = [p['name'] for p in search_results[:2]]
    return f"Found relevant products: {', '.join(product_names)}. {search_results[0]['description']}"

# Test with RAG
response = ask_ai_with_rag("What features does TurboCache Pro have?", products)
print("WITH RAG (Accurate Response):")
print("=" * 50)
print(response)

## Part 6: Compare Different Queries

Let's test how different search terms retrieve different context and affect the response.

In [ ]:
# Test queries that should match different products
test_queries = [
    "I need something fast",           # Should find TurboCache
    "I need security",                  # Should find SecureVault  
    "cloud backup solution",            # Should find CloudSync
    "real-time data processing",        # Should find DataFlow
    "quantum computing",                # Should find nothing
]

for query in test_queries:
    results = search_products(query, products, threshold=1)
    print(f"Query: '{query}'")
    print(f"Matches: {[r['name'] for r in results] if results else 'No matches'}")
    print("-" * 40)

## Part 7: Handle Edge Cases

A production RAG system needs to handle various edge cases gracefully.

In [ ]:
def ask_ai_with_rag_improved(question, products):
    """Improved RAG with better edge case handling"""
    
    search_results = search_products(question, products)
    
    if not search_results:
        # No results found - be honest about it
        available = [p['name'] for p in products]
        return f"""I couldn't find any products matching '{question}'. 
        
Available products: {', '.join(available)}
        
Please try different search terms or ask about a specific product."""
    
    if len(search_results) > 2:
        # Too many results - ask for clarification
        product_names = [p['name'] for p in search_results]
        return f"""Multiple products match your query: {', '.join(product_names)}. 
        
Please be more specific about which one you're interested in."""
    
    # Good match - provide detailed information
    context = create_context(search_results)
    return f"Based on our catalog:\n\n{context}"

# Test edge cases
print("Edge Case 1: No matches")
print(ask_ai_with_rag_improved("quantum blockchain AI", products))
print("\n" + "="*50 + "\n")

print("Edge Case 2: Too many matches")
print(ask_ai_with_rag_improved("data", products))
print("\n" + "="*50 + "\n")

print("Edge Case 3: Good match")
print(ask_ai_with_rag_improved("fast caching", products))

## Bonus Challenge: Improve Search Relevance

Try implementing more sophisticated search strategies.

In [ ]:
def advanced_search(query, products):
    """
    Implement a more sophisticated search:
    - Partial word matching (e.g., "sec" matches "security")
    - Check product names and descriptions too
    - Weight matches by location (name > keywords > description)
    """
    query_words = query.lower().split()
    matches = []
    
    for product in products:
        score = 0
        
        # Check product name (highest weight)
        for word in query_words:
            if word in product['name'].lower():
                score += 3
        
        # Check keywords (medium weight)
        for word in query_words:
            if any(word in kw for kw in product['keywords']):
                score += 2
        
        # Check description (lower weight)
        for word in query_words:
            if word in product['description'].lower():
                score += 1
        
        if score > 0:
            matches.append({'product': product, 'score': score})
    
    # Sort by score
    matches.sort(key=lambda x: x['score'], reverse=True)
    return [m['product'] for m in matches]

# Test advanced search
print("Basic search for 'sec':")
print([p['name'] for p in search_products("sec", products)])
print()
print("Advanced search for 'sec':")
print([p['name'] for p in advanced_search("sec", products)])

## Summary and Key Takeaways

### What We Learned

1. **RAG eliminates hallucinations** by grounding AI responses in real data
2. **Simple keyword search** can be surprisingly effective for structured data
3. **Context quality matters** - better search means better answers
4. **Edge case handling** is crucial for production systems

### When to Use RAG

✅ **Perfect for:**
- Product catalogs and documentation
- Company knowledge bases
- Dynamic or frequently changing data
- Domain-specific information

❌ **Not needed for:**
- General programming knowledge
- Well-known facts
- Creative writing tasks

### Next Steps

In production, you would:
1. Use semantic search with embeddings for better matching
2. Implement proper chunking for large documents
3. Add a vector database for scale
4. Monitor retrieval quality and user satisfaction
5. Implement feedback loops for continuous improvement